<a href="https://colab.research.google.com/github/ZikrullaRaxmatov/Zako_AI_Projects/blob/main/Spam_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install python-telegram-bot

In [ ]:
!pip install nest_asyncio

In [ ]:
!pip install telethon

In [ ]:
from telethon import TelegramClient
from telethon.tl.functions.channels import JoinChannelRequest
import nest_asyncio

api_id = 'api_id'
api_hash = 'api_hash'
chat = 'https://t.me/KOREAISHLAR777'
chat_name = chat.split('/')[3]
chat_messages = []

client = TelegramClient("ssesion_name", api_id, api_hash)
nest_asyncio.apply()

async def join_channel(client, channel_link):
  try:
    await client.JoinChannelRequest(channel_link)
    print(f"Successfully joined to {channel_link}")
  except:
    print(f"Failed to join {channel_link}")

async def scrape_message(client, channel, limit=100):

  async for message in client.iter_messages(channel, limit):
    if message.text:
      chat_messages.append(message.text)
      #print(message.text)
      #print('-'*40)

async def main():
  #await join_channel(client, chat)
  await scrape_message(client, chat, 1000)

async with client:
  client.loop.run_until_complete(main())

# We need a new DataFrame with the new contents
df_new_rows = pd.DataFrame({'text': chat_messages})

# Call Pandas.concat to create a new DataFrame from several existing ones
df = pd.concat([df, df_new_rows])
df.to_csv(f'{chat_name}.csv', index=False)
df.shape

(20147, 1)

In [ ]:
# Importing libraries for telegram
from typing import Final
from telegram import Update
from telegram.ext import Application, CommandHandler, MessageHandler, ContextTypes, filters
import nest_asyncio

# Importing libraries for ai model
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

import pandas as pd
import numpy as np

In [ ]:
# Loading a dataset

#data = pd.read_csv('./spam.csv', encoding='ISO-8859-1')
data = pd.read_csv('./spam_uz.csv' )
data

,text,target
0,Tabriklaymiz! Siz NBU tomonidan 1.000.000 (bir...,1
1,Tabriklaymiz! Siz Anor Banki tomonidan 1.000.0...,1
2,Tabriklaymiz! Siz Xalq Banki tomonidan 1.000.0...,1
3,Tabriklaymiz! Siz Aloqa Banki tomonidan 1.000....,1
4,Tabriklaymiz! Siz Asaka Banki tomonidan 1.000....,1
...,...,...
15467,CHINLEDA 1 HAFTALIK ISH BOR 2 AYOL KISHIGA SAM...,0
15468,Akalar ish bormi?!g'arbiy Tegu Manpyeong metro...,0
15469,Ваалейкум ассалом ва рахматуллоху ва баракатуху.,0
15470,ASSALOM ALAYKUM HOP AKA,0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15472 entries, 0 to 15471
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    15472 non-null  object
 1   target  15472 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 241.9+ KB


In [ ]:
data.duplicated().sum()

0

In [ ]:
# Separating features and targets
X = data['text']
y = data['target']

In [ ]:
# Converting text data into numbreic features
vectorizer = CountVectorizer()
X_vectorized = vectorizer.fit_transform(X)

In [ ]:
# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.22, random_state=0)

In [ ]:
# Creating a model
model = MultinomialNB()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

acc = accuracy_score(y_test, y_pred)
print(f"Accuracy: {acc}")

report = classification_report(y_test, y_pred)
print(f"Classification Report:\n {report}")

Accuracy: 0.9700352526439483
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.97      0.98      3361
           1       0.30      1.00      0.46        43

    accuracy                           0.97      3404
   macro avg       0.65      0.98      0.72      3404
weighted avg       0.99      0.97      0.98      3404



In [ ]:
def detect_spam(new_msg):
  #new_msg = ["Congratulations! You've won a $500 gift card, click here to claim"]
  new_msg_vectorized = vectorizer.transform([new_msg])
  predictions = model.predict(new_msg_vectorized)

  if predictions[0] == 1:
    print('This is a spam message!!!')
  else:
    print('Normal message')

detect_spam("Tabriklaymiz, siz Asaka bank tomonidan omadli ishtirokchi bo'ldiz va 5.000.000 so'm egasi bo'ldingiz. Sovg'ani olish uchun linkni bosing")

Normal message


In [ ]:
TOKEN: Final = '8095481386:AAHyVwCyVonMKFcTRJLEZ_UEVS8ZHFgMEMA'
BOT_USERNAME: Final = 'habar_tekshiruvchi_bot'

# Commands
async def start_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
  await update.message.reply_text("Hi, This is a 'sapm detector' bot!")

async def help_command(update: Update, context: ContextTypes.DEFAULT_TYPE):
  await update.message.reply_text("I can check only messages weather, 'spam' or 'non-spam'")

# Responses
def handle_response(text: str) -> str:
  lower_text: str = text.lower()

  new_msg_vectorized = vectorizer.transform([lower_text])
  predictions = model.predict(new_msg_vectorized)

  if predictions[0] == 'spam':
    return "Be careful, this may be a 'SPAM' message."


async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
  message_type: str = update.message.chat.type
  text: str = update.message.text

  print(f"User {update.message.chat.id} in {message_type}: '{text}'")

  '''
  if message_type == 'group':
    if BOT_USERNAME in text:
      new_text: str = text.replace(BOT_USERNAME, '').strip()
      response: str = handle_response(new_text)
  else:
  '''
  response: str = handle_response(text)

  print(f"Bot: {response}")
  await update.message.reply_text(response)


async def handle_error(update: Update, context: ContextTypes.DEFAULT_TYPE):
  print(f"Updating prosess: {update} caused error '{context.error}'")

if __name__ == '__main__':
  nest_asyncio.apply()
  print("Starting the bot...")
  app = Application.builder().token(TOKEN).build()

  # Commands
  app.add_handler(CommandHandler('start', start_command))
  app.add_handler(CommandHandler('help', help_command))

  # Messages
  app.add_handler(MessageHandler(filters.TEXT, handle_message))

  # Errors
  app.add_error_handler(handle_error)

  # Polls the bot
  print("Polling the bot...")
  app.run_polling(poll_interval=1)

Failed to join https://t.me/pishiriqlar


,text
0,3 soat qoldi🥳 Tez botga kirib raqam oling👇\n\n...
1,Faqat bugun raqam olganlar hisoblanadi\nBotga ...
0,3 soat qoldi🥳 Tez botga kirib raqam oling👇\n\n...
1,Faqat bugun raqam olganlar hisoblanadi\nBotga ...
2,200 ming kam pul emas\nQatnashdizmi?\nKim nech...
...,...
88,**Shirinlik 😍**\n\n__Muallif I mir.fatimas__\n...
89,**BANANCHIK SHIRINLIGI 😍**\n\n__Muallif | rece...
90,**Eng mazali chig’anoq shirinligi **😋\n\n__Mua...
91,**Choco gusiniy lapki**😋 \n\n__Muallif | aziza...
